In [1]:
import json

import pvl
import requests

## Create the necessary input data

To create an ISD we require the following:

- target_name, e.g. Mercury
- capture_date (PDS data product dates appear to be ISO 8601 compliant)
- instrument, e.g. mdis-nac
- focal_plane_temperature (optionally included for temperature dependent focal lengths)
- spacecraft_id, e.g. messenger
- spacecraft_clock_count, from the label
- exposure_duration, from the label
- lighttime_correction, to tell spice what to use
- min_elevation (make optional?)
- max_elevation (make optional?)

For the last two, it might be easier/better to take the intital center point from the PDS label and hit a low res. DEM to find these automatically for the user.

In [10]:
label = pvl.load('/data/cassini/iss/N1829241514_1.LBL')
data = {'target_name': label['TARGET_NAME'],
        'capture_date': label['START_TIME'].isoformat(),
        'instrument': label['INSTRUMENT_NAME'], 
        'spacecraft_id' : label['INSTRUMENT_HOST_NAME'],
        'spacecraft_clock_count': label['SPACECRAFT_CLOCK_START_COUNT'],
        'exposure_duration': label['EXPOSURE_DURATION'],
        'lighttime_correction':'LT+S', 
        'min_elevation': -100,
        'max_elevation': 100
        }
import json
data = json.dumps(data)

In [11]:
data

'{"target_name": "ENCELADUS", "capture_date": "2015-12-19T17:34:38.317000", "instrument": "IMAGING SCIENCE SUBSYSTEM - NARROW ANGLE", "spacecraft_id": "CASSINI ORBITER", "spacecraft_clock_count": "1829241512.246", "exposure_duration": 1500.0, "lighttime_correction": "LT+S", "min_elevation": -100, "max_elevation": 100}'

## Standard API landing page

In [5]:
r = requests.get('http://smalls:8002/api/1.0/')
r.json()

{'data': {'urls': {'missions': '/api/1.0/missions.'}}, 'success': True}

## Pushing the API into a namespace and versioning is good practice

In [5]:
r = requests.get('http://smalls:8002/api/1.0')
r.json()

{'data': {'urls': {'missions': '/api/1.0/missions.'}}, 'success': True}

## Example listing of the kernels

In [6]:
r = requests.get('http://smalls:8002/api/1.0/missions/cassini/metakernels')
r.json()

{'data': [{'id': 227,
   'mission': 'cassini',
   'name': 'cas_1997_v21.tm',
   'newest': True,
   'path': '/data/spice/co-s_j_e_v-spice-6-v1.0/cosp_1000/extras/mk/cas_1997_v21.tm',
   'year': '1997-01-01'},
  {'id': 228,
   'mission': 'cassini',
   'name': 'cas_1997_v20.tm',
   'newest': False,
   'path': '/data/spice/co-s_j_e_v-spice-6-v1.0/cosp_1000/extras/mk/cas_1997_v20.tm',
   'year': '1997-01-01'},
  {'id': 229,
   'mission': 'cassini',
   'name': 'cas_1997_v19.tm',
   'newest': False,
   'path': '/data/spice/co-s_j_e_v-spice-6-v1.0/cosp_1000/extras/mk/cas_1997_v19.tm',
   'year': '1997-01-01'},
  {'id': 230,
   'mission': 'cassini',
   'name': 'cas_1997_v18.tm',
   'newest': False,
   'path': '/data/spice/co-s_j_e_v-spice-6-v1.0/cosp_1000/extras/mk/cas_1997_v18.tm',
   'year': '1997-01-01'},
  {'id': 231,
   'mission': 'cassini',
   'name': 'cas_1997_v17.tm',
   'newest': False,
   'path': '/data/spice/co-s_j_e_v-spice-6-v1.0/cosp_1000/extras/mk/cas_1997_v17.tm',
   'year': '19

## Example generation of the ISD object

Notice that every endpoint returns with a `success` key and a `data` key at the top level.  This helps the user know if the call was successful and that all of the response is within the `data` key.

In [12]:
data

'{"target_name": "ENCELADUS", "capture_date": "2015-12-19T17:34:38.317000", "instrument": "IMAGING SCIENCE SUBSYSTEM - NARROW ANGLE", "spacecraft_id": "CASSINI ORBITER", "spacecraft_clock_count": "1829241512.246", "exposure_duration": 1500.0, "lighttime_correction": "LT+S", "min_elevation": -100, "max_elevation": 100}'

In [13]:
r = requests.post('http://smalls:8002/api/1.0/missions/cassini/csm_isd', data=data)
r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Use the ISD

In [10]:
import usgscam as cam
from cycsm.isd import Isd

In [11]:
isd = r.json()['data']['isd']

In [12]:
i = Isd.loads(isd)

In [13]:
plugin = cam.mdis.MdisPlugin()
camera = plugin.from_isd(i, plugin.modelname(0))

In [14]:
camera.imagesize

(1024.0, 1024.0)

In [32]:
xyz = camera.imageToGround(512, 512, 0)
xyz

[232745.39404384792, 108032.64063985727, 1416.4229696467519]

In [35]:
camera.name

'MDISNAC_USGSAstro_1_Linux64_csm30.so'

In [33]:
camera.groundToImage(0,0,0)

[512.0, 512.0]

## Random Messenger Image

The above could be perceived as being too canned.  Here, pull a random image from the PDS and instantiate a camera model.

In [14]:
from ftplib import FTP
import os
from random import choice

In [15]:
# Run only once to avoid too many open FTP connections
ftp = FTP('pdsimage2.wr.usgs.gov')
ftp.login()
dirs = ftp.nlst('archive/mess-e_v_h-mdis-2-edr-rawdata-v1.0/MSGRMDS_1001/DATA/')

In [19]:
data_dir = choice(dirs)
f = choice(ftp.nlst(data_dir))
fname = os.path.basename(f)
with open('/data/messenger_sample/{}'.format(fname), 'wb') as file:
    ftp.retrbinary('RETR {}'.format(f), file.write)

In [20]:
newfile = os.path.join('/data/messenger_sample/{}'.format(fname))

In [21]:
# Simple helper function to use the API
def create_data_pacakge(in_file):
    label = pvl.load(in_file)
    data = {'target_name': label['TARGET_NAME'],
            'capture_date': label['START_TIME'].isoformat(),
            'instrument': label['INSTRUMENT_NAME'], 
            'focal_plane_temperature': label['FOCAL_PLANE_TEMPERATURE'].value,
            'spacecraft_id' : label['INSTRUMENT_HOST_NAME'],
            'spacecraft_clock_count': label['SPACECRAFT_CLOCK_START_COUNT'],
            'exposure_duration': label['EXPOSURE_DURATION'].value,
            'lighttime_correction':'LT+S', 
            'min_elevation': -100,
            'max_elevation': 100,
            }
    return json.dumps(data)

print('Processing a new image: {}'.format(newfile))

# Call the func to create the data package for submission to the micro-service
data = create_data_pacakge(newfile)

# Call the micro-service
r = requests.post('http://localhost:5000/api/1.0/csm_isd', data=data)

# Get the ISD back and instantiate a local ISD for the image
isd = r.json()['data']['isd']
i = Isd.loads(isd)

# Create the plugin and camera as usual
plugin = cam.mdis.MdisPlugin()
camera = plugin.from_isd(i, plugin.modelname(0))

# Call I2G
xyz = camera.imageToGround(512.5, 512.5, 0)
print(xyz)

# Call G2I
camera.groundToImage(*xyz)

Processing a new image: /data/messenger_sample/EW1012946978D.IMG
[-722219.633387954, 583190.2862394641, 2255872.392486973]


[512.5000000007078, 512.4999999999983]